<a href="https://colab.research.google.com/github/O3Oa/MyStancodeProjects/blob/main/AI_projects/digits_classifier/digits_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define Project Folder
FOLDERNAME = 'Colab\ Notebooks/AI_projects/digits_classifier'

%cd drive/MyDrive/$FOLDERNAME

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/AI_projects/digits_classifier


# 新增區段

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define device
import torch
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print('Device:', device)

Device: cuda


In [ ]:
# Import dataset
import torchvision.datasets as dset
import torchvision.transforms as T


In [ ]:
# Show data image
train_data = dset.MNIST(root='./train', train=True, download=True)
train_data[0][0]

In [ ]:
# Load Existing Dataset
train_data = dset.MNIST(root='./train', train=True, download=True, transform=T.ToTensor())
val_data = dset.MNIST(root='./val', train=False, download=True, transform=T.ToTensor())

In [ ]:
# Check Data Dimension
print(train_data)
print(val_data)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./train
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset MNIST
    Number of datapoints: 10000
    Root location: ./val
    Split: Test
    StandardTransform
Transform: ToTensor()


In [ ]:
# Check processed input
train_data[0][0]

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

In [ ]:
num_train = len(train_data)
num_val = len(val_data)
print('Number of training:', num_train)
print('Number of validation:', num_val)

Number of training: 60000
Number of validation: 10000


Basic CNN model
accuracy: 96%


In [ ]:
# Build Model 1
import torch.nn as nn
model = nn.Sequential(
        # N x 1 x 28 x 28
        nn.Conv2d(1, 64, 3, 1, 1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        # N x 64 x 14 x 14
        nn.Flatten(),
        nn.Linear(64*14*14, 10)
        )


In [ ]:
# Move model to GPU
model = model.cuda()

In [ ]:
# Create Mini-batches
from torch.utils.data import DataLoader
mini_trains = DataLoader(train_data, batch_size=128, shuffle=True)
mini_vals = DataLoader(val_data, batch_size=128, shuffle=True)

In [ ]:
# Training Procedure
def train(num_epoch, model, mini_trains, mini_vals, device, loss_function, optimizer):
  for epoch in range(num_epoch):
    num_iters = 0
    for x, y in mini_trains:
      model.train()
      x = x.to(device)
      y = y.to(device)
      # x = x.squeeze()
      scores = model(x)
      loss = loss_function(scores, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if num_iters % 10 == 0:
        evaluate_predictor(model, epoch, mini_vals, device)
      num_iters += 1

In [ ]:
# Validating Procedure
def evaluate_predictor(model, epoch, mini_vals, device):
  model.eval()
  with torch.no_grad():
    acc_count = 0
    for x, y in mini_vals:
      x = x.to(device)
      y = y.to(device)
      # x = x.squeeze()
      scores=model(x)
      predictions=scores.max(1)[1]
      acc = predictions.eq(y).sum().item()
      acc_count += acc
    print(f'Epoch[{epoch+1}] Acc: {acc_count/num_val}')

In [ ]:
# Define loss function & optimizer
import torch.optim as optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# Start training
train(1, model, mini_trains, mini_vals, device, loss_function, optimizer)

Epoch[1] Acc: 0.1506
Epoch[1] Acc: 0.7281
Epoch[1] Acc: 0.8383
Epoch[1] Acc: 0.8926
Epoch[1] Acc: 0.9009
Epoch[1] Acc: 0.9209
Epoch[1] Acc: 0.934
Epoch[1] Acc: 0.9326
Epoch[1] Acc: 0.9381
Epoch[1] Acc: 0.9347
Epoch[1] Acc: 0.9439
Epoch[1] Acc: 0.9522
Epoch[1] Acc: 0.9472
Epoch[1] Acc: 0.9555
Epoch[1] Acc: 0.9578
Epoch[1] Acc: 0.9551
Epoch[1] Acc: 0.953
Epoch[1] Acc: 0.9577
Epoch[1] Acc: 0.9509
Epoch[1] Acc: 0.95
Epoch[1] Acc: 0.9604
Epoch[1] Acc: 0.9632
Epoch[1] Acc: 0.9581
Epoch[1] Acc: 0.9628
Epoch[1] Acc: 0.9661
Epoch[1] Acc: 0.9667
Epoch[1] Acc: 0.9667
Epoch[1] Acc: 0.9658
Epoch[1] Acc: 0.9617
Epoch[1] Acc: 0.9621
Epoch[1] Acc: 0.9621
Epoch[1] Acc: 0.9666
Epoch[1] Acc: 0.9619
Epoch[1] Acc: 0.968
Epoch[1] Acc: 0.9642
Epoch[1] Acc: 0.9692
Epoch[1] Acc: 0.9556
Epoch[1] Acc: 0.9686
Epoch[1] Acc: 0.9593
Epoch[1] Acc: 0.9693
Epoch[1] Acc: 0.9684
Epoch[1] Acc: 0.9675
Epoch[1] Acc: 0.9702
Epoch[1] Acc: 0.9684
Epoch[1] Acc: 0.9729
Epoch[1] Acc: 0.9704
Epoch[1] Acc: 0.9628


In [ ]:
x = torch.zeros([128,1,28,28]).cuda()
model(x)

tensor([[ 0.0044,  0.0121, -0.0016,  ...,  0.0112, -0.0097,  0.0050],
        [ 0.0044,  0.0121, -0.0016,  ...,  0.0112, -0.0097,  0.0050],
        [ 0.0044,  0.0121, -0.0016,  ...,  0.0112, -0.0097,  0.0050],
        ...,
        [ 0.0044,  0.0121, -0.0016,  ...,  0.0112, -0.0097,  0.0050],
        [ 0.0044,  0.0121, -0.0016,  ...,  0.0112, -0.0097,  0.0050],
        [ 0.0044,  0.0121, -0.0016,  ...,  0.0112, -0.0097,  0.0050]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

RNN model
accuracy:91%

In [ ]:
# Model 2
class MyRNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super().__init__()
    self.lstm = nn.LSTM(input_size, hidden_size, batch_first = True)

    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    out,(h_n, c_n) = self.lstm(x)
    out = out[:, -1, :] #就是h_n
    out = self.fc(out)
    return out

model = MyRNN(28, 64, 10)

In [ ]:
# Move model to GPU
model = model.cuda()

In [ ]:
# Create Mini-batches
from torch.utils.data import DataLoader
mini_trains = DataLoader(train_data, batch_size=128, shuffle=True)
mini_vals = DataLoader(val_data, batch_size=128, shuffle=True)

In [ ]:
# Training Procedure
def train(num_epoch, model, mini_trains, mini_vals, device, loss_function, optimizer):
  for epoch in range(num_epoch):
    num_iters = 0
    for x, y in mini_trains:
      model.train()
      x = x.to(device)
      y = y.to(device)
      x = x.squeeze()
      scores = model(x)
      loss = loss_function(scores, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if num_iters % 10 == 0:
        evaluate_predictor(model, epoch, mini_vals, device)
      num_iters += 1

In [ ]:
# Validating Procedure
def evaluate_predictor(model, epoch, mini_vals, device):
  model.eval()
  with torch.no_grad():
    acc_count = 0
    for x, y in mini_vals:
      x = x.to(device)
      y = y.to(device)
      x = x.squeeze()
      scores=model(x)
      predictions=scores.max(1)[1]
      acc = predictions.eq(y).sum().item()
      acc_count += acc
    print(f'Epoch[{epoch+1}] Acc: {acc_count/num_val}')

In [ ]:
# Define loss function & optimizer
import torch.optim as optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# Start training
train(1, model, mini_trains, mini_vals, device, loss_function, optimizer)

Epoch[1] Acc: 0.1014
Epoch[1] Acc: 0.0952
Epoch[1] Acc: 0.1289
Epoch[1] Acc: 0.2285
Epoch[1] Acc: 0.304
Epoch[1] Acc: 0.3845
Epoch[1] Acc: 0.4374
Epoch[1] Acc: 0.4911
Epoch[1] Acc: 0.5426
Epoch[1] Acc: 0.5964
Epoch[1] Acc: 0.6227
Epoch[1] Acc: 0.6705
Epoch[1] Acc: 0.6801
Epoch[1] Acc: 0.7082
Epoch[1] Acc: 0.7492
Epoch[1] Acc: 0.746
Epoch[1] Acc: 0.7773
Epoch[1] Acc: 0.7838
Epoch[1] Acc: 0.8005
Epoch[1] Acc: 0.7925
Epoch[1] Acc: 0.8274
Epoch[1] Acc: 0.8162
Epoch[1] Acc: 0.8111
Epoch[1] Acc: 0.838
Epoch[1] Acc: 0.8478
Epoch[1] Acc: 0.8499
Epoch[1] Acc: 0.8537
Epoch[1] Acc: 0.8751
Epoch[1] Acc: 0.8639
Epoch[1] Acc: 0.8671
Epoch[1] Acc: 0.8763
Epoch[1] Acc: 0.8727
Epoch[1] Acc: 0.8702
Epoch[1] Acc: 0.8691
Epoch[1] Acc: 0.8737
Epoch[1] Acc: 0.8891
Epoch[1] Acc: 0.8859
Epoch[1] Acc: 0.8894
Epoch[1] Acc: 0.8862
Epoch[1] Acc: 0.9062
Epoch[1] Acc: 0.8979
Epoch[1] Acc: 0.9026
Epoch[1] Acc: 0.8994
Epoch[1] Acc: 0.9132
Epoch[1] Acc: 0.9075
Epoch[1] Acc: 0.9102
Epoch[1] Acc: 0.9068
